# Tokenização
Este notebook tem como objetivo aplicar vetorização e tokenização de enunciados e alternativas por meio de embeddings de palavras. O processo inclui etapas de pré-processamento textual, geração dos vetores e organização dos resultados para uso posterior em análises ou em modelos de aprendizado de máquina.

## Apoio

In [1]:
# Importando bibliotecas
import pandas as pd
import numpy as np
import re
from gensim.models import KeyedVectors

In [39]:
# Leitura do dataset
df_enem = pd.read_csv('/content/drive/MyDrive/PCD3/enem_data.csv')
df_enem.head(10)

,numero_questao,gabarito,NU_PARAM_A,nu_param_B,NU_PARAM_C,ANO,enunciado,alternativas
0,1,C,3.43894,0.97831,0.10855,2017,"No império africano do Mali, no século XIV, To...",A: isolamento geográﬁco do Saara ocidental; B...
1,2,D,3.00837,0.49169,0.13877,2017,Após a Declaração Universal dos Direitos Human...,A: ataque feito pelos japoneses à base milita...
2,3,D,0.60432,3.25992,0.08798,2017,"A moralidade, Bentham exortava, não é uma ques...",A: fundamentação cientíﬁca de viés positivist...
3,4,E,1.85031,0.57925,0.11344,2017,Fala-se muito nos dias de hoje em direitos do ...,A: modernização da educação escolar; B: atuali...
4,5,C,2.46290,0.76307,0.17672,2017,Na Constituição da República Federativa do Bra...,A: etnia e miscigenação racial; B: sociedade...
5,6,A,2.04543,0.74139,0.20814,2017,Descrição da fotografia: É retratada uma mulhe...,A: ambiguidade do trabalho doméstico exercido ...
6,7,D,4.02261,0.96775,0.17554,2017,A conﬁguração do espaço urbano da região do En...,A: expansão vertical; B: polarização naciona...
7,8,B,1.33884,2.80117,0.16665,2017,"México, Colômbia, Peru e Chile decidiram segui...",A: promover a livre circulação de trabalhadore...
8,9,E,3.15884,0.92459,0.09500,2017,Descrição da tabela: Indica a tipologia de áre...,A: erosão laminar; B: intemperismo físico; C...
9,10,A,0.67598,0.55190,0.00798,2017,O desgaste acelerado sempre existirá se o agri...,A: Elevação da acidez; B: Ampliação da salin...


In [40]:
# Função auxiliar para extrair as alternativas corretas de cada questão
def extrair_alternativa_correta(linha):
    letra = linha["gabarito"]
    alternativas = linha["alternativas"]

    # Regex para capturar o texto após "B:" até o próximo ponto e vírgula
    padrao = rf"{letra}:\s*(.*?)(?=;|$)"
    resultado = re.search(padrao, alternativas)
    return resultado.group(1).strip() if resultado else None

In [41]:
df_enem["alternativa_correta"] = df_enem.apply(extrair_alternativa_correta, axis=1)
df_enem.head(10)

,numero_questao,gabarito,NU_PARAM_A,nu_param_B,NU_PARAM_C,ANO,enunciado,alternativas,alternativa_correta
0,1,C,3.43894,0.97831,0.10855,2017,"No império africano do Mali, no século XIV, To...",A: isolamento geográﬁco do Saara ocidental; B...,posição relativa nas redes de circulação
1,2,D,3.00837,0.49169,0.13877,2017,Após a Declaração Universal dos Direitos Human...,A: ataque feito pelos japoneses à base milita...,execução de judeus e eslavos presos em guetos ...
2,3,D,0.60432,3.25992,0.08798,2017,"A moralidade, Bentham exortava, não é uma ques...",A: fundamentação cientíﬁca de viés positivist...,racionalidade de caráter pragmático
3,4,E,1.85031,0.57925,0.11344,2017,Fala-se muito nos dias de hoje em direitos do ...,A: modernização da educação escolar; B: atuali...,universalização do princípio da igualdade civil
4,5,C,2.46290,0.76307,0.17672,2017,Na Constituição da República Federativa do Bra...,A: etnia e miscigenação racial; B: sociedade...,espaço e sobrevivência cultural
5,6,A,2.04543,0.74139,0.20814,2017,Descrição da fotografia: É retratada uma mulhe...,A: ambiguidade do trabalho doméstico exercido ...,ambiguidade do trabalho doméstico exercido pel...
6,7,D,4.02261,0.96775,0.17554,2017,A conﬁguração do espaço urbano da região do En...,A: expansão vertical; B: polarização naciona...,segregação socioespacial
7,8,B,1.33884,2.80117,0.16665,2017,"México, Colômbia, Peru e Chile decidiram segui...",A: promover a livre circulação de trabalhadore...,fomentar a competitividade no mercado externo
8,9,E,3.15884,0.92459,0.09500,2017,Descrição da tabela: Indica a tipologia de áre...,A: erosão laminar; B: intemperismo físico; C...,recarga dos aquíferos
9,10,A,0.67598,0.55190,0.00798,2017,O desgaste acelerado sempre existirá se o agri...,A: Elevação da acidez; B: Ampliação da salin...,Elevação da acidez


## Word Embedding com Word2Vec

In [6]:
# Carregando o modelo de embedding
model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/PCD3/cbow_s300.txt')

In [24]:
# Dicionário auxiliar que armazena palavras já encontradas
vetor_cache = {}

# Função auxiliar que irá retornar os vetores de palavras
def generate_word2vec_embeddings(data, model, dim=300):
  not_found_words = set()
  embeddings = []

  for item in data:
    word_vectors = []
    words = re.sub(r'[^\w\s]', '', item.lower()).split()

    for word in words:
      try:
        if word in vetor_cache:
          word_vectors.append(vetor_cache[word])
        else:
          vetor_cache[word] = model[word]
          word_vectors.append(model[word])
      except KeyError:
        not_found_words.add(word)

    if word_vectors:
          # Calcula a média dos vetores
          mean_vector = np.mean(word_vectors, axis=0)
          embeddings.append(mean_vector)
    else:
          # Se não encontrou nenhuma palavra, adiciona um vetor nulo
          embeddings.append(np.zeros(model.vector_size))

  return embeddings, not_found_words


### Vetorização dos enunciados

In [ ]:
df_enem["enunciado_embeddings_word2vec"], not_found_words_enunciado = (
    generate_word2vec_embeddings(df_enem["enunciado"], model)
)

In [43]:
len(not_found_words_enunciado)

1885

### Vetorização das alternativas corretas

In [ ]:
# substituir o que é gabarito pelos distratores
df_enem["gabarito_embeddings_word2vec"], not_found_words_gabarito = (
    generate_word2vec_embeddings(df_enem["alternativa_correta"], model)
)

In [45]:
len(not_found_words_gabarito)

81

In [46]:
df_enem.head(10)

,numero_questao,gabarito,NU_PARAM_A,nu_param_B,NU_PARAM_C,ANO,enunciado,alternativas,alternativa_correta,enunciado_embbedings_word2vec,gabarito_embbedings_word2vec
0,1,C,3.43894,0.97831,0.10855,2017,"No império africano do Mali, no século XIV, To...",A: isolamento geográﬁco do Saara ocidental; B...,posição relativa nas redes de circulação,"[-0.030405086, 0.022974016, -0.027739665, -0.0...","[-0.12797333, -0.114192665, 0.13532682, -0.057..."
1,2,D,3.00837,0.49169,0.13877,2017,Após a Declaração Universal dos Direitos Human...,A: ataque feito pelos japoneses à base milita...,execução de judeus e eslavos presos em guetos ...,"[-0.03320758, 0.03166411, -0.04888896, 0.01524...","[-0.18022183, 0.041319773, -0.06070747, -0.069..."
2,3,D,0.60432,3.25992,0.08798,2017,"A moralidade, Bentham exortava, não é uma ques...",A: fundamentação cientíﬁca de viés positivist...,racionalidade de caráter pragmático,"[-0.0063855667, 0.022534302, -0.014108029, -0....","[-0.049211252, 0.073626995, 0.013877999, 0.085..."
3,4,E,1.85031,0.57925,0.11344,2017,Fala-se muito nos dias de hoje em direitos do ...,A: modernização da educação escolar; B: atuali...,universalização do princípio da igualdade civil,"[-0.014004647, 0.02424562, -0.010280999, 0.010...","[-0.06145667, 0.129228, -0.004840499, 0.033785..."
4,5,C,2.46290,0.76307,0.17672,2017,Na Constituição da República Federativa do Bra...,A: etnia e miscigenação racial; B: sociedade...,espaço e sobrevivência cultural,"[-0.013194435, 0.01645982, -0.0011703374, -0.0...","[0.045796998, -0.120854005, 0.06641175, -0.123..."
5,6,A,2.04543,0.74139,0.20814,2017,Descrição da fotografia: É retratada uma mulhe...,A: ambiguidade do trabalho doméstico exercido ...,ambiguidade do trabalho doméstico exercido pel...,"[-0.046363685, 0.03250221, 0.0001613248, 0.021...","[-0.020870503, 0.0074404515, -0.0058519007, -0..."
6,7,D,4.02261,0.96775,0.17554,2017,A conﬁguração do espaço urbano da região do En...,A: expansão vertical; B: polarização naciona...,segregação socioespacial,"[-0.03929502, -0.010324988, -0.010705049, -0.0...","[-0.085658, 0.108736, 0.044327497, -0.08828749..."
7,8,B,1.33884,2.80117,0.16665,2017,"México, Colômbia, Peru e Chile decidiram segui...",A: promover a livre circulação de trabalhadore...,fomentar a competitividade no mercado externo,"[-0.004068679, -0.0009967892, 0.02537945, 0.01...","[0.03787, -0.003211337, 0.22549248, -0.0773151..."
8,9,E,3.15884,0.92459,0.09500,2017,Descrição da tabela: Indica a tipologia de áre...,A: erosão laminar; B: intemperismo físico; C...,recarga dos aquíferos,"[-0.03472777, 0.009183877, 0.00088933244, -0.0...","[-0.13293, -0.0043570003, -0.228469, -0.192839..."
9,10,A,0.67598,0.55190,0.00798,2017,O desgaste acelerado sempre existirá se o agri...,A: Elevação da acidez; B: Ampliação da salin...,Elevação da acidez,"[0.0099083735, -0.011485817, -0.0075787096, -0...","[0.040357, -0.05295233, -0.061065663, -0.07836..."


## Salvando os datasets

In [ ]:
# df_enem.to_csv('/../data/final/enem_data_embeddings.csv')